In [1]:
import optparse, sys, os
from collections import namedtuple
import random
import math

import numpy as np

import bleu

In [2]:
class opts:
    reference = os.path.join("data", "train.en")
    nbest = os.path.join("data", "train.nbest")

translation_candidate = namedtuple("candidate", "sentence, inverse_scores, features")
ref = [line.strip().split() for line in open(opts.reference)]

In [50]:
nbests = []
for n, line in enumerate(open(opts.nbest)):
    (i, sentence, features) = line.strip().split("|||")
    (i, sentence) = (int(i), sentence.strip())
    features = np.array([float(it) for it in features.split()])
    if len(ref) <= i:
        break

    while len(nbests) <= i:
        nbests.append([])

    scores = tuple(bleu.bleu_stats(sentence.split(), ref[i]))
    inverse_scores = tuple([-x for x in scores])
    smoothed_score = bleu.smoothed_bleu(inverse_scores)

    nbests[i].append((translation_candidate(sentence, inverse_scores, features), smoothed_score))

    if n % 2000 == 0:
        print('loaded %d lines' % n)

    # small size for testing, delete it when release
    if n > 10000:
        break

loaded 0 lines
loaded 2000 lines
loaded 4000 lines
loaded 6000 lines
loaded 8000 lines
loaded 10000 lines


In [51]:
tau = 5000
alpha = 0.1
xi = 100
eta = 0.1
epochs = 5
theta = np.array([1.0 / len(features) for _ in range(len(features))])

In [58]:
mistakes = 0
discount = 0.7
for i in range(epochs):
    for nbest in nbests:

        def get_sample():
            sample = []
            for i in range(tau):
                # s1, s2 = random.sample(nbest, 2)
                s1 = random.choice(nbest)
                s2 = random.choice(nbest)
                if math.fabs(s1[1] - s2[1]) > alpha:
                    if s1[1] > s2[1]:
                        sample.append((s1, s2))
                    else:
                        sample.append((s2, s2))
                else:
                    continue
            return sample

        samples = sorted(get_sample(), key=lambda s: s[0][1] - s[1][1], reverse=True)
        samples = samples[:xi]
        
        # grad = np.zeros_like(theta)
        pre_grad = np.zeros_like(theta)
        
        for s in samples:
            s1 = s[0][0]
            s2 = s[1][0]
            if np.dot(theta, s1.features) <= np.dot(theta, s2.features):
                mistakes += 1
                
                grad = s1.features - s2.features
                pre_grad = pre_grad * 0.1 + grad * eta
                # print pre_grad
                theta += grad * eta
                
        del(samples)
    print "epoch", 
    print i

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4


In [59]:
mistakes, theta

(2846, array([ 0.19636667,  0.16666667, -0.03333333,  0.16666667,  0.36666667,
         0.24506667]))

In [49]:
for i in theta:
    print i

1.98241355471
0.0965299639668
-1.82244868587
0.0965299639668
-9.01896975028
-3.38351481355
